<a href="https://colab.research.google.com/github/zdwhite/Thinkful-Unit-2/blob/master/Challenge_Iterate_and_evaluate_your_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#installs the latest seaborn update to colab runtime
!pip install -U seaborn

    100% |████████████████████████████████| 215kB 25.0MB/s 
  Found existing installation: seaborn 0.7.1
    Uninstalling seaborn-0.7.1:
      Successfully uninstalled seaborn-0.7.1


In [0]:
#This restarts the Python runtime so any modules that get installed/upgraded automatically get imported
import os
os.kill(os.getpid(), 9)

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re as re

In [0]:
# Grab and process the raw data.
data_path = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/yelp_labelled.txt"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['message', 'sentiment']
sms_raw['message']=sms_raw['message'].str.lower()

### Words of caution

A lot of this code is a repition of it self. I did not spend much time creating a function to shorten the length .

 Create 5 different Classifiers (prediction models)
 

1.   Give an output of performance for each model
2.   Do any of your classifiers seem to overfit?
3.   Which seem to perform the best? Why?
4.   Which features seemed to be most impactful to performance?



In [0]:



#before we tackle the NB model Let's get a sense of some key words

#Split
good_sentiment=sms_raw[sms_raw['sentiment']==1]['message'].apply(lambda x:x.split())

#Looking through our bag of words, there are lots of punctuations and possibly special characters let's get rid of those
good_sentiment = pd.DataFrame(good_sentiment).apply(lambda x: pd.Series(x['message']),axis=1).stack().reset_index(level=1, drop=True).reset_index()

good_sentiment.columns=['Index','Words']

good_sentiment['Words']=good_sentiment['Words'].apply(lambda x: re.sub('[^A-Za-z0-9]+','',x))

GSWC = pd.DataFrame(good_sentiment['Words'].value_counts().reset_index())
GSWC['sentiment']='Good'

#Split
bad_sentiment=sms_raw[sms_raw['sentiment']==0]['message'].apply(lambda x:x.split())

bad_sentiment = pd.DataFrame(bad_sentiment).apply(lambda x: pd.Series(x['message']),axis=1).stack().reset_index(level=1, drop=True).reset_index()

bad_sentiment.columns=['Index','Words']

#Combine
#Looking through our bag of words, there are lots of punctuations and possibly special characters let's get rid of those
bad_sentiment['Words']=bad_sentiment['Words'].apply(lambda x: re.sub('[^A-Za-z0-9]+','',x))

bad_sentiment['Words'].value_counts()

BSWC = pd.DataFrame(bad_sentiment['Words'].value_counts().reset_index())
BSWC['sentiment']='Bad'

Bag_of_words=GSWC.append(BSWC)
Bag_of_words.columns=['Word','Count','Sentiment']



In [0]:
#implement outer join

#pd.merge(adf, bdf,how='outer', on='x1')

Good= Bag_of_words[Bag_of_words["Sentiment"] == "Good"]
Bad = Bag_of_words[Bag_of_words["Sentiment"] == "Bad"]

test = pd.merge(Good,Bad,how='outer',on='Word',suffixes =["_Good","_Bad"]).set_index('Word')

#test.fillna(0,inplace=True)


In [0]:
############# CLASSIFIER 1 ( The dumb version)
#In this version instead of filtering keywords we are going to grab words that are in bad or good
### Output ###
# Confusion Matrix for both good and bad predictions
# Accuracy for both

###https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
##### Recursive Feature selection

In [0]:
# all keywords that appear to have bad sentiment
## Bad Key words

keywords = test[test.iloc[:,2]>0].iloc[:,2].index.values

#create features in sms_raw
for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

#create data for NB classifier
#Classifies Sentiment column into True for Bad sentiment and False for Good sentiment
sms_raw['sentiment'] = (sms_raw['sentiment'] == 0)
data = sms_raw[keywords]
target = sms_raw['sentiment']



#run the classifier through the Bernoulli Naive Bayes model    
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))
from sklearn.metrics import confusion_matrix
confusion_matrix(target, y_pred)

Number of mislabeled points out of a total 1000 points : 221


array([[478,  22],
       [199, 301]])

In [0]:
# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

With 20% Holdout: 0.59
Testing on Sample: 0.779


In [0]:
from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

array([0.68, 0.62, 0.7 , 0.64, 0.61, 0.59, 0.57, 0.66, 0.59, 0.64])

As we can see from the hold out tests and the cross validation accuracy scores this model is not very accurate.

Lets check the converse for Good seniment words and see how it performs

In [0]:
################# RE-RUN same model using good sentiment keywords and show accuracy
# Grab and process the raw data.
data_path = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/yelp_labelled.txt"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['message', 'sentiment']
sms_raw['message']=sms_raw['message'].str.lower()



#### Re-create the same classifier but using Good keywords

## Good Keywords
keywords = test[test.iloc[:,0]>0].iloc[:,0].index.values

#create features in sms_raw
for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

#create data for NB classifier
#Classifies Sentiment column into True for Bad sentiment and False for Good sentiment
sms_raw['sentiment'] = (sms_raw['sentiment'] == 1)
data = sms_raw[keywords]
target = sms_raw['sentiment']



#run the classifier through the Bernoulli Naive Bayes model    
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

from sklearn.metrics import confusion_matrix
confusion_matrix(target, y_pred)

Number of mislabeled points out of a total 1000 points : 163


array([[476,  24],
       [139, 361]])

In [0]:
# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

With 20% Holdout: 0.63
Testing on Sample: 0.837


In [0]:
from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

array([0.68, 0.66, 0.69, 0.66, 0.65, 0.65, 0.65, 0.71, 0.61, 0.7 ])

We notice that the cross validation of the dummy classifier for Bad Keywords performs fairly similarly to that of using good seniment keywords

BAD Seniment Key word Accuracy array = [0.68, 0.62, 0.7 , 0.64, 0.61, 0.59, 0.57, 0.66, 0.59, 0.64]

GOOD Seniment Key word Accuracy array = [0.68, 0.66, 0.69, 0.66, 0.65, 0.65, 0.65, 0.71, 0.61, 0.7 ]

Let's try a smarter way of chosing keywords

In [0]:
#### This classifier filters out keywords by making sure their count is larger than the other sentiment count
# This will filter for Bad Keywords

#Bag_of_words.describe()
#std = Bag_of_words.describe()['Count'][2]

keywords = []
for word in test.index.values:
  #print(test[test['Word']==word])
  #if test.loc[word][0] < test.loc[word][2]+std*2:
  if test.loc[word][0] < test.loc[word][2]:
    keywords.append(word)
  #if (test.loc[word][0]) == 0 & (test.loc[word][2] > 2):
    #if word not in keyword:
      #keyword.append(word)

# Grab and process the raw data.
data_path = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/yelp_labelled.txt"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['message', 'sentiment']
sms_raw['message']=sms_raw['message'].str.lower()

#create features in sms_raw
for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

#create data for NB classifier
#Classifies Sentiment column into True for Bad sentiment and False for Good sentiment
sms_raw['sentiment'] = (sms_raw['sentiment'] == 0)
data = sms_raw[keywords]
target = sms_raw['sentiment']



#run the classifier through the Bernoulli Naive Bayes model    
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(target, y_pred))

from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

Number of mislabeled points out of a total 1000 points : 300
With 20% Holdout: 0.575
Testing on Sample: 0.7
[[413  87]
 [213 287]]


array([0.71, 0.63, 0.7 , 0.57, 0.59, 0.66, 0.6 , 0.67, 0.58, 0.66])

The model got worse ... that's progress we all can get behind.

In [0]:
#### This classifier filters out keywords by making sure their count is larger than the other sentiment count
# This will filter for Good Keywords

#Bag_of_words.describe()
#std = Bag_of_words.describe()['Count'][2]

keywords = []
for word in test.index.values:
  #print(test[test['Word']==word])
  #if test.loc[word][0] < test.loc[word][2]+std*2:
  if test.loc[word][0] > test.loc[word][2]:
    keywords.append(word)
  #if (test.loc[word][0]) == 0 & (test.loc[word][2] > 2):
    #if word not in keyword:
      #keyword.append(word)

# Grab and process the raw data.
data_path = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/yelp_labelled.txt"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['message', 'sentiment']
sms_raw['message']=sms_raw['message'].str.lower()

#create features in sms_raw
for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

#create data for NB classifier
#Classifies Sentiment column into True for Bad sentiment and False for Good sentiment
sms_raw['sentiment'] = (sms_raw['sentiment'] == 1)
data = sms_raw[keywords]
target = sms_raw['sentiment']



#run the classifier through the Bernoulli Naive Bayes model    
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

Number of mislabeled points out of a total 1000 points : 253
With 20% Holdout: 0.61
Testing on Sample: 0.747


array([0.55, 0.6 , 0.64, 0.6 , 0.61, 0.59, 0.68, 0.59, 0.62, 0.66])

The model keeps getting worse! Let's take the top 10 most frequently used words in good or bad as long as their frequency is higher than the other

In [0]:
#### This classifier filters out keywords by making sure their count is larger than the other sentiment count
# This will filter for Bad Keywords

#Bag_of_words.describe()
#std = Bag_of_words.describe()['Count'][2]

keywords = []
i=0
for word in test.index.values:
  #print(test[test['Word']==word])
  #if test.loc[word][0] < test.loc[word][2]+std*2:
  if test.loc[word][2]>test.loc[word][0] :
    keywords.append(word)
    i +=1
  if i==len(test.index.values):
    break


# Grab and process the raw data.
data_path = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/yelp_labelled.txt"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['message', 'sentiment']
sms_raw['message']=sms_raw['message'].str.lower()

#create features in sms_raw
for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

#create data for NB classifier
#Classifies Sentiment column into True for Bad sentiment and False for Good sentiment
sms_raw['sentiment'] = (sms_raw['sentiment'] == 0)
data = sms_raw[keywords]
target = sms_raw['sentiment']



#run the classifier through the Bernoulli Naive Bayes model    
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(target, y_pred))

from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

Number of mislabeled points out of a total 1000 points : 300
With 20% Holdout: 0.575
Testing on Sample: 0.7
[[413  87]
 [213 287]]


array([0.71, 0.63, 0.7 , 0.57, 0.59, 0.66, 0.6 , 0.67, 0.58, 0.66])

This makes sense we have limited keywords for features, the difference between 10 features and 100 features isn't much different.

Before we pull in all the data sets, lets try a smart approach at filtering keyword features and add in a feature to look for words that are capitalized.



In [0]:
#### New classifier using two data sets for generating keywords tested against our yelp data-set


########## Yelp Reviews
data_path1 = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/yelp_labelled.txt")
sms_raw1 = pd.read_csv(data_path1, delimiter= '\t', header=None)
sms_raw1.columns = ['message', 'sentiment']

########## Amazon reviews

### Something in this data set isn't allowing me to 
data_path2 = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/amazon_cells_labelled.txt")

sms_raw2 = pd.read_csv(data_path2, delimiter= '\t', header=None)
sms_raw2.columns = ['message', 'sentiment']

sms_raw2['message']=sms_raw2['message'].str.lower()

########## IMDB reviews
data_path3 = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/imdb_labelled.txt")

sms_raw3 = pd.read_csv(data_path3, delimiter= '\t', header=None)
sms_raw3.columns = ['message', 'sentiment']

sms_raw3['message']=sms_raw3['message'].str.lower()


sms_raw = pd.concat([sms_raw1,sms_raw3])
#sms_raw = sms_raw1

In [0]:



#before we tackle the NB model Let's get a sense of some key words

#Split
good_sentiment=sms_raw[sms_raw['sentiment']==1]['message'].apply(lambda x:x.split())

#Looking through our bag of words, there are lots of punctuations and possibly special characters let's get rid of those
good_sentiment = pd.DataFrame(good_sentiment).apply(lambda x: pd.Series(x['message']),axis=1).stack().reset_index(level=1, drop=True).reset_index()

good_sentiment.columns=['Index','Words']

good_sentiment['Words']=good_sentiment['Words'].apply(lambda x: re.sub('[^A-Za-z0-9]+','',x))

GSWC = pd.DataFrame(good_sentiment['Words'].value_counts().reset_index())
GSWC['sentiment']='Good'

#Split
bad_sentiment=sms_raw[sms_raw['sentiment']==0]['message'].apply(lambda x:x.split())

bad_sentiment = pd.DataFrame(bad_sentiment).apply(lambda x: pd.Series(x['message']),axis=1).stack().reset_index(level=1, drop=True).reset_index()

bad_sentiment.columns=['Index','Words']

#Combine
#Looking through our bag of words, there are lots of punctuations and possibly special characters let's get rid of those
bad_sentiment['Words']=bad_sentiment['Words'].apply(lambda x: re.sub('[^A-Za-z0-9]+','',x))

bad_sentiment['Words'].value_counts()

BSWC = pd.DataFrame(bad_sentiment['Words'].value_counts().reset_index())
BSWC['sentiment']='Bad'

Bag_of_words=GSWC.append(BSWC)
Bag_of_words.columns=['Word','Count','Sentiment']



In [0]:
#implement outer join

#pd.merge(adf, bdf,how='outer', on='x1')

Good= Bag_of_words[Bag_of_words["Sentiment"] == "Good"]
Bad = Bag_of_words[Bag_of_words["Sentiment"] == "Bad"]

test = pd.merge(Good,Bad,how='outer',on='Word',suffixes =["_Good","_Bad"]).set_index('Word')

test.fillna(0,inplace=True)


In [0]:
#### Let's use our filtering classifier and generate our bad sentiment keywords
# This will filter for Bad Keywords

#Bag_of_words.describe()
std = Bag_of_words.describe()['Count'][2]

keywords = []
for word in test.index.values:
  #print(test[test['Word']==word])
  #if test.loc[word][0] < test.loc[word][2]+std*2:
  if test.loc[word][2] >test.loc[word][0] :
    keywords.append(word)


# Grab and process the raw data.
sms_raw=sms_raw1

#create features in sms_raw
for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

#create data for NB classifier
#Classifies Sentiment column into True for Bad sentiment and False for Good sentiment
sms_raw['sentiment'] = (sms_raw['sentiment'] == 0)
data = sms_raw[keywords]
target = sms_raw['sentiment']



#run the classifier through the Bernoulli Naive Bayes model    
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(target, y_pred))

from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

Number of mislabeled points out of a total 1000 points : 229
With 20% Holdout: 0.665
Testing on Sample: 0.771
[[470  30]
 [199 301]]


array([0.74, 0.67, 0.76, 0.63, 0.6 , 0.64, 0.66, 0.73, 0.64, 0.71])

##Do any of your classifiers seem to overfit? 
>None of them

##Which seem to perform the best? Why?
>It's splitting hairs on which model performs the best when comparing their cross validation arrays

##Which features seemed to be most impactful to performance?
>I'm unsure which features seem to be the most impactful in terms of keyword selection.

>I don't know why there is no improvement other than the fact that individual key words aren't all that great as features for NB models.





In [0]:
#####################################IGNORE CODE BELOW THIS LINE 
#----------------------------------------------------------------------------

In [0]:
###### I can't remember why I coded this TBH for now it's depretiated code until proven otherwise

#df = sms_raw
#code 1 & 0 to Good and Bad

df = pd.DataFrame([])
#df.loc[df['sentiment']==1,'sentiment'],df.loc[df['sentiment']==0,'sentiment'] = 'Good','Bad'
df['message']=sms_raw['message'].apply(lambda x:x.split())
df['sentiment']=sms_raw['sentiment']

#Bag_of_words = pd.DataFrame([],columns=['Word','Count','Sentiment'])



#df = pd.DataFrame([])
#df.loc[df['sentiment']==1,'sentiment'],df.loc[df['sentiment']==0,'sentiment'] = 'Good','Bad'
df['message']=sms_raw['message'].apply(lambda x:x.split()) 

df=df.set_index('sentiment',append=True).apply(lambda x: pd.Series(x['message']),axis=1).stack()
df=pd.DataFrame(df.reset_index().reset_index())

df=df.drop(['index','level_0','level_2'],axis=1)
df.columns=['sentiment','messages']

In [0]:
###### Attempts to make a big ass function that I could just call for modeling not worth the effort ATM
#assumes data is precoded to 0 or 1
# 
#def model ()

#inputs ( data, messages ,key)
# split data

#def split_data(data,messages,key):
#  # good == 1  ,bad == 0
#  Bag_of_words = pd.DataFrame([],columns=['Word','Count','Sentiment'])
#  df=data
#  
#  df.loc[df['sentiment']==1,'sentiment'],df.loc[df['sentiment']==0,'sentiment'] = 'Good','Bad'
#  df[messages]=df[messages].apply(lambda x:x.split())
#  test=df.apply(lambda x: pd.DataFrame(x),axis=1).stack().reset_index(level=1, drop=True).reset_index()
#  return test